# WIP

In [1]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

In [2]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")
model.config.forced_decoder_ids = None

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

Found cached dataset librispeech_asr_dummy (/Users/Tony/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [3]:
from undecorated import undecorated
from types import MethodType

generate_with_grad = undecorated(model.generate)
model.generate_with_grad = MethodType(generate_with_grad, model)

In [4]:
outputs = model.generate_with_grad(input_features, num_beams=3, num_return_sequences=1, output_scores=True, return_dict_in_generate=True)
outputs.keys()

/opt/homebrew/Caskroom/miniforge/base/envs/ml-pytorch-speech/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


odict_keys(['sequences', 'sequences_scores', 'scores', 'beam_indices'])

In [5]:
outputs.sequences

tensor([[50257, 50362,  1770,    13,  2264,   346,   353,   318,   262, 46329,
           286,   262,  3504,  6097,    11,   290,   356,   389,  9675,   284,
          7062,   465, 21443,    13, 50256]])

In [6]:
outputs.scores

(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
         [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], grad_fn=<CopySlices>),
 tensor([[-11.7886,     -inf,     -inf,  ..., -15.2796, -16.0519, -18.6459],
         [-11.7886,     -inf,     -inf,  ..., -15.2796, -16.0519, -18.6459],
         [-11.7886,     -inf,     -inf,  ..., -15.2796, -16.0519, -18.6459]],
        grad_fn=<CopySlices>),
 tensor([[ -8.1542,     -inf,     -inf,  ..., -14.6152, -14.8889, -17.6750],
         [ -9.6977,     -inf,     -inf,  ..., -12.0022, -12.2440, -13.8926],
         [ -8.7922,     -inf,     -inf,  ..., -14.3252, -14.1414, -17.0619]],
        grad_fn=<CopySlices>),
 tensor([[-12.3110,     -inf,     -inf,  ..., -12.4940, -12.6871, -14.0868],
         [-17.9017,     -inf,     -inf,  ..., -15.1163, -16.6716, -16.3644],
         [-17.7269,     -inf,     -inf,  ..., -14.7390, -16.2557, -16.1587]],
        grad_fn=<CopySlices>),
 tensor([[-18.5015,     -inf

In [7]:
outputs.scores[0].grad_fn